In [1]:

import pandas as pd
import json
import numpy as np
import pylab
import nltk
import operator 
import json
from collections import Counter
import regex as re

import matplotlib.pyplot as plt

json_filename='C:/Users/Robin/Dropbox/Travaux_&_Rapports_Stages/UbicomLab-GATech/MeToo/code/balancetonporc_com/strories_balancetonporc_com.json'
df_story = pd.read_json(json_filename, orient="columns") #we have gather 1500 story from the website
 

In [2]:
import re
import numpy as np
remove_letter=lambda x: re.sub("[^0-9]", "",x)
df_story['nb-views']=df_story['nb-views'].map(remove_letter)
pd.to_numeric(df_story['nb-views'], errors='coerce').astype(np.int64)
#view=df_story['nb-views'].tolist()
view=[]
for i in range(len(df_story['nb-views'])):
    view.append(int(df_story['nb-views'].iloc[i]))
print(df_story['nb-views'].mean())
print(np.max(view))

inf
7618


In [3]:
import unicodedata

df_story['text'] = df_story['text'].str.replace('’', ' ')
df_story['text'] = df_story['text'].str.replace("""'""", ' ')

#supprimer les accents
def delete_accent(x):
    x1 = unicode(x,'iso-8859-1')
    x=unicodedata.normalize('NFD', x1).encode('ascii', 'ignore')
def strip_accents(s):
    return(''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn'))
    
df_story['text']=df_story['text'].map(strip_accents)    

#remove number
find_numbers = re.compile(r'\d+')
find_ages = re.compile(r'\d(.)an[s]+')
revome_numbers=lambda x: re.sub(find_numbers, '', x)
revome_ages=lambda x: re.sub(find_ages, '', x)

df_story['text']=df_story['text'].map(revome_ages)
df_story['text']=df_story['text'].map(revome_numbers)


doc_stories=df_story['text'].values.tolist()

print(doc_stories[:5])

['Humiliation, isolement, coups, insultes… Quand je voulais un peu de liberte, il me fallait son autorisation, ce n etait jamais sans contreparties sexuelles, que je sois d accord ou non. Plus le temps passait, plus ses exigences etaient tordues… me prendre dans la douleur l excitait… J ai toujours tout cache pour les enfants, sans savoir qu il faisait la meme chose avec mes sœurs et ma niece. Je suis restee trente ans avec ce porc. Ma plainte pour toutes ces violences a etait classee sans suite et pourtant j ai ete vue par un legiste. Ce porc n a meme jamais ete interroge sur ce qu il ma fait subir, quelle horreur ! Justice pour qui ? Pas pour moi en tout cas.', 'J ai grandi dans un petit village de Dordogne peuple de  habitants. Voici un petit resume des salopards que j ai croises entre mes  et mes  – c est-a-dire durant mes toutes premieres annees de vie, entre la toute petite enfance et l adolescence.° Entre mes  et mes  et demi, ma mere me  depose, chaque matin, chez une «\xa0noun

## Cleaning data

In [4]:
import stop_words
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.corpus import stopwords

#print(stop_words.get_stop_words('fr'))
ajout=["""’""",'…','a','j','d','t','etait','ete','tres','etais','les','dis','dit','car','en','ca','ça','cest','jétais','ant','âgé','jetais','netais', 'pa', 'an','nétait','ca','qu','âge','age','qil','quil','ans','quelle','plus',"""javais""",r"""[sc]est""",'ct','jai',]
stop = stop_words.get_stop_words('fr')+ajout
#print(stop)                                                                                           
                                                                                                                           
exclude = set(string.punctuation + '»'+'«'+"’"+"""'"""+'…'+'–'+'°') 
lemma = WordNetLemmatizer()


def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_stories_clean = [clean(story).split() for story in doc_stories]
print(doc_stories_clean[:5])


[['humiliation', 'isolement', 'coup', 'insultes', 'voulais', 'liberte', 'fallait', 'autorisation', 'jamais', 'contreparties', 'sexuelles', 'accord', 'non', 'temp', 'passait', 'exigences', 'etaient', 'tordues', 'prendre', 'douleur', 'excitait', 'toujours', 'cache', 'enfants', 'savoir', 'faisait', 'meme', 'chose', 'sœurs', 'niece', 'restee', 'trente', 'porc', 'plainte', 'toutes', 'violence', 'classee', 'suite', 'pourtant', 'vue', 'legiste', 'porc', 'meme', 'jamais', 'interroge', 'subir', 'horreur', 'justice', 'ca'], ['grandi', 'petit', 'village', 'dordogne', 'peuple', 'habitant', 'voici', 'petit', 'resume', 'salopards', 'croises', 'entre', 'c', 'estadire', 'durant', 'toutes', 'premiere', 'annees', 'vie', 'entre', 'toute', 'petite', 'enfance', 'adolescence', 'entre', 'demi', 'mere', 'depose', 'matin', 'chez', 'nounou', 'agree', 'madame', 'genevieve', 'b', 'riberac', 'dordogne', 'soir', 'declare', 'mere', 'william', 'montre', 'fess', 'jeune', 'souvenir', 's', 'passe', 'exactement', 'flippe

### save a text file

In [23]:
import os
newpath = r'C:\Users\Robin\Stories' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

for i in range(len(doc_stories_clean)):
    file = open('Stories\story'+str(i)+'.txt','w',encoding='utf-8') 
    for j in range(len(doc_stories_clean[i])):
        text=doc_stories_clean[i][j]
        file.write(text+' ')
    file.write('\n')
    file.close()
 
 


## Use of Gensim

In [27]:
# Importing Gensim, # code importé depuis https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim

from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_stories_clean)
dictionary.save('dictionary_btp1.dict')
dictionary.filter_extremes(no_below=20, no_above=0.1)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(story) for story in doc_stories_clean]
corpora.MmCorpus.serialize('corpus_btp1.mm', doc_term_matrix)
#print(doc_term_matrix)
#drop the extreme
print(dictionary))

print(len(doc_term_matrix))

1499
1499


In [125]:
print(len(dictionary))

17635


In [126]:
from time import time
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO,
                   filename='running.log',filemode='w')

## Runnin LDA model

In [137]:
start=time()
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=20, id2word = dictionary, passes=200)
# k found here : https://www.quora.com/Latent-Dirichlet-Allocation-LDA-What-is-the-best-way-to-determine-k-number-of-topics-in-topic-modeling
print('used: {:.2f}s'.format(time()-start))


used: 1620.18s


## Save the model

In [138]:
ldamodel.save('topic_btp2.model')


## plot it

In [139]:
from gensim.models import LdaModel

import pyLDAvis.gensim
import gensim

loading = LdaModel.load('topic_btp2.model')
pyLDAvis.enable_notebook()

In [140]:
d = gensim.corpora.Dictionary.load('dictionary_btp1.dict')
c = gensim.corpora.MmCorpus('corpus_btp1.mm')
lda = gensim.models.LdaModel.load('topic_btp2.model')

In [142]:
print(c)

MmCorpus(1499 documents, 17635 features, 116778 non-zero entries)


In [154]:
cm = gensim.CoherenceModel(model=lda, corpus=c, coherence='u_mass')  # tm is the trained topic model
cm.get_coherence()

AttributeError: module 'gensim' has no attribute 'CoherenceModel'

In [141]:
data = pyLDAvis.gensim.prepare(lda, c, d)
data

C:\Users\Robin\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      44.515077        1       1 -0.192140  0.035830
5      13.808914        1       2 -0.103147 -0.070966
3       4.712945        1       3 -0.081580 -0.019417
17      3.712441        1       4 -0.091324  0.028080
14      2.958745        1       5 -0.008157 -0.073065
18      2.944808        1       6  0.043166 -0.002266
7       2.928278        1       7 -0.077012  0.046928
9       2.812288        1       8 -0.039381 -0.030718
4       2.570670        1       9  0.034714 -0.094088
13      2.558732        1      10 -0.039588 -0.028851
16      2.520098        1      11 -0.047135  0.076140
1       2.277427        1      12  0.042120 -0.089219
19      1.946397        1      13  0.022241  0.014970
15      1.938785        1      14  0.054197  0.027939
12      1.664667        1      15  0.082794 -0.019543
0       1.662949        1      16  0.049499  0.164689
2       1.269759        1      17  0.096906  0.013169
10      1.158508        1      18  0.076395 -0.034475
8       1.104849        1      19  0.084010  0.024146
11      0.933663        1      20  0.093422  0.030717, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
385   Default  365.000000         femmes  365.000000  30.0000  30.0000
906   Default  702.000000           bien  702.000000  29.0000  29.0000
8     Default  591.000000         jamais  591.000000  28.0000  28.0000
168   Default  639.000000            moi  639.000000  27.0000  27.0000
501   Default  508.000000          fille  508.000000  26.0000  26.0000
26    Default  729.000000           meme  729.000000  25.0000  25.0000
305   Default  553.000000           etre  553.000000  24.0000  24.0000
67    Default  359.000000           mere  359.000000  23.0000  23.0000
97    Default  435.000000          femme  435.000000  22.0000  22.0000
124   Default  531.000000          homme  531.000000  21.0000  21.0000
70    Default  521.000000           chez  521.000000  20.0000  20.0000
586   Default  310.000000        famille  310.000000  19.0000  19.0000
120   Default  181.000000        voiture  181.000000  18.0000  18.0000
368   Default  299.000000         hommes  299.000000  17.0000  17.0000
270   Default  573.000000          apres  573.000000  16.0000  16.0000
732   Default  380.000000         autres  380.000000  15.0000  15.0000
236   Default  511.000000           puis  511.000000  14.0000  14.0000
142   Default  616.000000           rien  616.000000  13.0000  13.0000
432   Default  405.000000           dire  405.000000  12.0000  12.0000
32    Default  310.000000           porc  310.000000  11.0000  11.0000
33    Default  271.000000        plainte  271.000000  10.0000  10.0000
411   Default  395.000000           peur  395.000000   9.0000   9.0000
61    Default  422.000000            vie  422.000000   8.0000   8.0000
1014  Default  296.000000           pere  296.000000   7.0000   7.0000
12    Default  369.000000            non  369.000000   6.0000   6.0000
564   Default  433.000000           main  433.000000   5.0000   5.0000
21    Default  399.000000       toujours  399.000000   4.0000   4.0000
257   Default  319.000000         parent  319.000000   3.0000   3.0000
264   Default  394.000000           jour  394.000000   2.0000   2.0000
258   Default  296.000000          porte  296.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
2764  Topic20    2.286499            loi   30.537257   2.0819  -6.3342
2708  Topic20    4.155844         bureau  150.927841   1.0815  -5.7367
305   Topic20    6.285644           etre  553.669934   0.1955  -5.3229
1156  Topic20    2.442190          doigt   42.859858   1.8088  -6.2683
1158  Topic20    2.868073          ainsi   71.010582   1.4646  -6.1076
399   Topic20    3.280910            ose  105.523168   1.2030  -5.9731


## Adjust the parameter


In [ ]:
import tmtoolkit
#https://github.com/WZBSocialScienceCenter/tmtoolkit

## Improve the model with n-gramm or sentence

In [ ]:
from nltk.util import ngrams

for doc in docs:
    docs[doc] = docs[doc] + ["_".join(w) for w in ngrams(docs[doc], 2)]

In [151]:
from nltk import bigrams
story_bigram_doc=[]
for i in range(len(doc_stories_clean)):
    for j in range(len(doc_stories_clean[i])):
        story_bigram_doc.append(doc_stories_clean[i][j])
    
story_bigram=bigrams(story_bigram_doc)
freq_bigram= Counter(story_bigram)
print(freq_bigram.most_common(50))

[(('aujourd', 'hui'), 327), (('porter', 'plainte'), 89), (('chez', 'moi'), 86), (('porte', 'plainte'), 83), (('quelque', 'chose'), 81), (('petite', 'fille'), 77), (('quelques', 'annees'), 59), (('harcelement', 'sexuel'), 49), (('dire', 'non'), 44), (('agression', 'sexuelle'), 43), (('main', 'fess'), 43), (('plusieurs', 'reprises'), 42), (('chez', 'lui'), 41), (('gros', 'porc'), 41), (('meme', 'temp'), 40), (('agressions', 'sexuelles'), 38), (('annees', 'tard'), 36), (('mal', 'aise'), 36), (('rien', 'dire'), 36), (('jamais', 'rien'), 35), (('jamais', 'parle'), 35), (('rien', 'dit'), 35), (('plusieurs', 'annees'), 34), (('quelques', 'jours'), 34), (('quelques', 'mois'), 34), (('pendant', 'annees'), 32), (('chez', 'parent'), 31), (('jeunes', 'fille'), 30), (('toute', 'vie'), 30), (('jeune', 'homme'), 29), (('plainte', 'contre'), 29), (('jusqu', 'jour'), 29), (('mi', 'main'), 28), (('ose', 'dire'), 28), (('contre', 'mur'), 27), (('derriere', 'moi'), 26), (('balance', 'porc'), 26), (('bien'

In [152]:
from nltk import trigrams 
tweet_trigram = trigrams(story_bigram_doc)
freq_trigram= Counter(tweet_trigram)
print(freq_trigram.most_common(20))

[(('quelques', 'annees', 'tard'), 14), (('rien', 'ose', 'dire'), 13), (('porter', 'plainte', 'contre'), 11), (('quelques', 'jours', 'tard'), 11), (('pendant', 'plusieurs', 'annees'), 10), (('porte', 'plainte', 'contre'), 10), (('quelques', 'mois', 'tard'), 10), (('mi', 'main', 'fess'), 9), (('plaque', 'contre', 'mur'), 8), (('quelques', 'jours', 'apres'), 8), (('rentrer', 'chez', 'moi'), 8), (('insiste', 'insiste', 'insiste'), 8), (('victime', 'agression', 'sexuelle'), 8), (('jamais', 'parle', 'personne'), 7), (('aujourd', 'hui', 'sais'), 7), (('jusqu', 'aujourd', 'hui'), 7), (('ose', 'porter', 'plainte'), 6), (('tant', 'bien', 'mal'), 6), (('devant', 'chez', 'moi'), 6), (('jamais', 'rien', 'dit'), 6)]
